In [9]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()
from __future__ import print_function
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


# the f(x) part of differential equation y'(x) = f(X)
def funct(x, y):
    return 1 + (y * (2 * x + 1) / (math.pow(x, 2)))

# function that computes exaxt solution by initial values
def exact(X, x0, y0):
    # list for exact solutions
    YE = []
    # compute constant for initial value problem
    C = ((y0 + math.pow(x0, 2)) * math.exp(1 / x0)) / math.pow(x0, 2)
    for x in X:
        YE.append(((C * math.pow(x, 2)) / (math.exp(1 / x))) - math.pow(x, 2))
    return YE

# euler numerical method
# for finding approximated solutions
def euler(X, y0, step):
    yn = y0
    Y = [y0]
    for x in X:
        yn = yn + step * funct(x, yn)
        Y.append(yn)
    # exculude the last element of the list 
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]


def improved_euler(X, y0, step):
    yni = y0
    Y = [yni]
    for x in X:
        k1 = funct(x, yni)
        k2 = funct(x + step, yni + (step * k1))
        yni = yni + ((step / 2) * (k1 + k2))
        Y.append(yni)
    # exculude the last element of the list 
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]


def runge_kutta(X, y0, step):
    ynr = y0
    Y = [ynr]
    for x in X:
        rk1 = funct(x, ynr)
        rk2 = funct(x + (step / 2), ynr + ((step / 2) * rk1))
        rk3 = funct(x + (step / 2), ynr + ((step / 2) * rk2))
        rk4 = funct(x + step, ynr + step * rk3)
        ynr = ynr + (step / 6) * (rk1 + 2 * rk2 + 2 * rk3 + rk4)
        Y.append(ynr)
    # exculude the last element of the list 
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]

# function that computes error of method
# by substacting approximated result from exaxt one 
def local_error(Y, YE):
    ER = []
    for i in range(len(Y)):
        ER.append(math.fabs(Y[i] - YE[i]))
    return ER

def global_error(x0, y0, x, P):
    ERU =[]
    ERI =[]
    ERR =[]
    for p in P:
        h = (float) (x - x0)/p
        X = np.arange(x0, x + h, h)
        YE = exact(X, x0, y0)
        Y = euler(X, y0, h)
        ERI.append(max(local_error(Y, YE)))
        YI = improved_euler(X, y0, h)
        ERU.append(max(local_error(YI, YE)))
        YR = runge_kutta(X, y0, h)
        ERR.append(max(local_error(YR, YE)))
    return ERU, ERI, ERR
        
        
    
    
# function that plots the graphs of approximation of each method 
# and errors that gives that methods
def plot_graph(X, YE, Y, YI, YR, N, ERU, ERI, ERR):
    fig = plt.figure(1, figsize=(10, 10))

    #plot figure of all 3 methods with exact on one graph
    sub = fig.add_subplot(211)
    sub.set_title("Approximated solutions")
    sub.plot(X, YE, 'r', label="Exact")
    sub.plot(X, Y, 'g', label="Euler")
    sub.plot(X, YI, 'b', label="Improved")
    sub.plot(X, YR, 'y', label="Runge Kutta")
    sub.set_xlabel("x")
    sub.set_ylabel("y")
    sub.legend()

    #plot an error of each method on one graph
    sub = fig.add_subplot(212)
    sub.set_title("Global errors")
    sub.plot(N, ERU, 'g', label="Euler")
    sub.plot(N, ERI, 'b', label="Improved")
    sub.plot(N, ERR, 'y', label="Runge Kutta")
    sub.set_xlabel("x")
    sub.set_ylabel("dy")
    sub.legend()

    plt.show()


#function that calls all other functions
def main(x, step, init_x, init_y, n0, N):
    x0 = int(init_x)
    y0 = int(init_y)
    b = int(x)
    h = float(step)
    
    X = np.arange(x0, b + h, h)
    YE = exact(X, x0, y0)
    Y = euler(X, y0, h)
    YI = improved_euler(X, y0, h)
    YR = runge_kutta(X, y0, h)
    
    P = np.arange(int(n0), int(N) + 1, 1)
    ERU, ERI, ERR = global_error(x0,y0, b, P)
    
    plot_graph(X, YE, Y, YI, YR, P, ERU, ERI, ERR)


# makes plots interactive
if __name__ == '__main__':
    interact(main, init_x='1', init_y='1', x='10', step='1', n0='9', N='90')


interactive(children=(Text(value='10', description='x'), Text(value='1', description='step'), Text(value='1', …